In [89]:
#pip install selenium
from selenium import webdriver
#pip install beautifulsoup4
from bs4 import BeautifulSoup
# tentando melhorar a questao dos bloqueios
#pip install undetected-chromedriver==2.1.1   
import undetected_chromedriver as uc 
from time import sleep
import re
import csv


In [90]:
#array que vai conter todos os dados da pesquisa
data = []
americanasUrlBase = 'https://www.americanas.com.br'

In [91]:
#configurando o webdrive
options = webdriver.ChromeOptions() 
options.add_argument("start-maximized")
#o executable path está apenas com esse caminho porque se encontra na mesma pasta que o projeto
# uma alternativa é colocar no path do sistema
navegador = uc.Chrome(options=options, executable_path=r'../chromedriver.exe') 

In [92]:
#entra no site da americanas e navega até a primeira pagina de pesquisa da busca
def gettingPageOne(search):
    navegador.get(americanasUrlBase)
    sleep(2)
    barraPesquisa = navegador.find_element('xpath', '//*[@id="rsyswpsdk"]/div/header/div[1]/div[1]/div/div[1]/form/input')
    barraPesquisa.send_keys(search)
    barraPesquisa.submit()
    sleep(2)

In [93]:
def gettinData():
    pagina = BeautifulSoup(navegador.page_source, 'html.parser') 

    # o findAll pega todos as divs que tenham a classe passada. perceba a importancia do uso do regex com o beatifulsoup
    #  se você for no site da americanas e clicar em inspecionar elemento, vai ver que os produtos tem uma classe estranha
    # mas que sempre mantem o padrão de inStockCard__Wrapper.
    
    allDivs = pagina.findAll('div',attrs={'class':re.compile('(inStockCard__Wrapper)')}) 

    #aqui eu pego dentro de todas as divs, as informaçoes que me interessam, perceba mais uma vez o uso do regex
    for div in allDivs:
        data.append({'name':div.find('h3', attrs={'class':re.compile('(product-name__Name)')}).text,
        'price': div.find('span', attrs={'class':re.compile('(src__Text-sc)')}).text,
        'link':americanasUrlBase+div.find('a').get('href')
         })

In [94]:
#função responsavel por passar para a prox pagina
def goNextPage():
    try:
        sleep(4)
        # nao sei se por o botao de next está fora da tela ou o js dele nao ter renderizado, essa função da erro caso eu nao 
        #use o script para fazer a janela da scroll até o ponto onde ela é renderizada 
        navegador.execute_script("window.scrollTo(0, window.scrollY + 4400)")
        sleep(4)
        buttonNext = navegador.find_element('xpath', '//*[@id="rsyswpsdk"]/div/main/div/div[3]/div[3]/div/ul/li[10]')
        #note que o selenium tenta simular um ser humano mexendo no navegador, se tiver algo na frente do botão como um popup 
        #impossibilitando o click manualmente, o selenium tbm nao vai conseguir clicar e dará um erro
        buttonNext.click()
        return True
    except:
        print('erro')
        return False

In [95]:
def getscv(search): 
    header = ['name', 'price','link']
    
    with open('americanas'+search+'.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames = header)
        writer.writeheader()
        writer.writerows(data) 

In [96]:
# #o correto seria ao inves de fazer o print da data colocar em algo como um csv, mas já que esse nao é o foco fiquei com 
# #preguiça, dps faço
# def SeeData():
#     index = 1 # so pra eu saber a qtd de informaçoes que ele achou :)
#     for dado in data:
#         print(index)
#         index+=1
#         print('nome: ', dado['name'])
#         print('preço: ', dado['price'])
#         print('link: ', dado['link'])
#         print('\n')

In [97]:
def closePopup():
    button =navegador.find_element('xpath', '//*[@id="rsyswpsdk"]/div/header/div[2]/button')
    button.click()

In [98]:
#função que executa a busca 
from logging import NullHandler


def searchAmericanas(search, numberPages):
    page = 1
    gettingPageOne(search)
    try:
        closePopup()
    except:
        NullHandler
    while page <= numberPages:
        gettinData()
        goNext = goNextPage()
        if(goNext):
            page+=1
        else:
            break
    
    navegador.close()
    # SeeData()
    getscv(search)

In [99]:
searchAmericanas('televisao',3)